In [1]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
import neptune 

# print torch version
print(torch.__version__)

/home/lirannoc/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/lirannoc/.conda/envs/ltsf6/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.1.1+cu118


In [2]:

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')
# seed
parser.add_argument('--seed', type=int, default=2021, help='random seed')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
#parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--root_path', type=str, default='/cs/cs_groups/azencot_group/TSlib/dataset/ETT-small/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# MTLinear
parser.add_argument('--cluster_dist', type=float, default=0.5, help='distance threshhold for clustering, d = 1-cos(alpha). 1 returns one cluster, 0 returns n clusters')
parser.add_argument('--layer_type', type=str, default="DLinear", help='later type for MTLinear, supports: DLinear, NLinear , Linear, RLinear')
parser.add_argument('--multi_early_stopping', type=bool, default=False, help='multi early stopping for each group')
parser.add_argument('--use_horizon_penalty', type=int, default=0 ,help='use the horzion dimension for the loss penalty')
parser.add_argument('--use_variates_penalty', type=int, default=0 ,help='use the variates dimension for the loss penalty')
parser.add_argument('--penalty_param', type=int, default=1 ,help=' penalty parameter, the larger the stronger the penalty')
parser.add_argument('--reproduce_original', type=int, default=1, help='reproduce the original results, ensuring the same random, this is not necessary for the model to work')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# Neptune todo
parser.add_argument('--project', type=str, default="azencot-group/Ablation-MultiTask", help='neptune project')
parser.add_argument('--token', type=str, default="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJh\
cGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlZDU5ZjRiOC04NjljLTQ1NGYtOTFjMi01ODRjZjBjNWE1Mj\
gifQ==", help='neptune api token')

   # example ili Linear
args = parser.parse_args("\
         --is_training 1 \
         --root_path ./dataset/illness/ \
         --data_path national_illness.csv \
         --model_id ili \
         --model Linear \
         --data custom \
         --features M \
         --seq_len 106 \
         --label_len 18 \
         --pred_len 60 \
         --e_layers 2 \
         --d_layers 1 \
         --factor 3 \
         --enc_in 7 \
         --dec_in 7 \
         --c_out 7 \
         --des 'Exp' \
         --itr 1 \
         --learning_rate 0.01\
         --batch_size 32".split()) 

args = parser.parse_args("\
         --is_training 1 \
         --root_path ./dataset/illness/ \
         --data_path national_illness.csv \
         --model_id ili \
         --model DLinear \
         --data custom \
         --features M \
         --seq_len 36 \
         --label_len 18 \
         --pred_len 60 \
         --enc_in 7 \
         --dec_in 7 \
         --c_out 7 \
         --des 'Exp' \
         --itr 1 \
         --learning_rate 0.01\
         --batch_size 32".split()) 



args = parser.parse_args("\
         --is_training 1 \
         --root_path ./dataset/illness/ \
         --data_path national_illness.csv \
         --model_id ili \
         --model MTLinear \
         --data custom \
         --features M \
         --seq_len 36 \
         --label_len 18 \
         --pred_len 60 \
         --enc_in 7 \
         --dec_in 7 \
         --c_out 7 \
         --des 'Exp' \
         --itr 1 \
         --learning_rate 0.01\
         --layer_type NLinear\
         --cluster_dist 0.5\
         --use_horizon_penalty 1\
         --use_variates_penalty 1\
         --multi_early_stopping 1\
         --penalty_param 2\
         --batch_size 32".split()) 

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False


In [3]:
args.model = "MTLinear"
args.cluster_dist = 0.134
args.penalty_param = 2
args.layer_type = "DLinear"
args.multi_early_stopping= 1
args.train_epochs = 20
args.seed = 2021
args.use_horizon_penalty = 1
args.use_variates_penalty = 1
args.pred_len = 24

In [4]:


if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Exp = Exp_Main

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_lt{}_pp{}_cd{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des,
            args.layer_type,
            args.penalty_param,
            args.cluster_dist, ii)
        
        run = neptune.init_run(project=args.project, api_token=args.token,git_ref=False)
        

        random.seed(args.seed+ii)
        torch.manual_seed(args.seed+ii)
        np.random.seed(args.seed+ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        if not args.train_only:
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting)
            #test todo
            folder_path = './results/' + setting + '/'
            metrics = np.load(folder_path + 'metrics.npy', allow_pickle = True)
            run['metrics'] = {"mae": metrics[0], "mse": metrics[1]}

        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)

        torch.cuda.empty_cache()
        run.stop()


else:
    ii = 0
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                  args.model,
                                                                                                  args.data,
                                                                                                  args.features,
                                                                                                  args.seq_len,
                                                                                                  args.label_len,
                                                                                                  args.pred_len,
                                                                                                  args.d_model,
                                                                                                  args.n_heads,
                                                                                                  args.e_layers,
                                                                                                  args.d_layers,
                                                                                                  args.d_ff,
                                                                                                  args.factor,
                                                                                                  args.embed,
                                                                                                  args.distil,
                                                                                                  args.des, ii)

    exp = Exp(args)  # set experiments

    if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)
    else:
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting, test=1)
    torch.cuda.empty_cache()


Args in experiment:
Namespace(is_training=1, train_only=False, model_id='ili', model='MTLinear', seed=2021, data='custom', root_path='./dataset/illness/', data_path='national_illness.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=36, label_len=18, pred_len=24, individual=False, embed_type=0, enc_in=7, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, cluster_dist=0.134, layer_type='DLinear', multi_early_stopping=1, use_horizon_penalty=1, use_variates_penalty=1, penalty_param=2, reproduce_original=1, num_workers=10, itr=1, train_epochs=20, batch_size=32, patience=3, learning_rate=0.01, des="'Exp'", loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False, project='azencot-group/Ablation-MultiTask', token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYX